## Julia GASS

Testing the GASS with Julia

In [15]:
using PyCall
using Distributions
using Statistics

using Distances , Random 
using DataFrames
using Printf

import CSV

rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"

push!(LOAD_PATH,"$rootdir/master/src")
using GASS

import PyPlot

@pyimport astroquery.vizier as vizier
@pyimport astropy.units as u
@pyimport astropy.coordinates as coord

## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

In [16]:
## Types ##

struct _observation
    Array_Configuration_File::String
    Result_Folder::String
    Display_Verbose::Bool
    Observatory_Latitude::Float64
    Source_Declination::Float64
    Source_Hour_Angle::Float64
    Subarray_Number::Int
end

struct _subarrayParameters
        Pads_Per_Subarray::Vector{Int}
        Subarray_Name::Vector{String}
        Subrange::Array{UnitRange{Int64},1}
        Spatial_Resolution::Vector{Float64}
        Maximum_Recoverable_Scale::Vector{Float64}
        Elongation::Vector{Float64}
        Sidelobe_Level::Vector{Float64}
end

struct _weight
    Weight_Subarray::Vector{Float64}
    Weight_Spatial_Resolution::Vector{Float64}
    Weight_Maximum_Recoverable_Scale::Vector{Float64}
    Weight_Elongation::Vector{Float64}                  
    Weight_Sidelobe_Levels::Vector{Float64}
end

struct _GA
    Number_Iterations::Int
    Population_Size::Int
    Termination_Condition::Bool
    Threshold::Float64
    Mutation_Rate::Float64
    Tournament_Size::Int
    Number_Elitism::Int
end

struct _cfg
    arr::AbstractDataFrame 
    obs::_observation
    sub::_subarrayParameters
    wei::_weight
    ga::_GA
end

In [17]:
function _input_parameters(file)
    let
        param = []
        open(file,"r") do f
            for line in eachline(f)
                l = lstrip(line)
                if length(l) > 0 && l[1] != '#'
                    res = strip.(split(l,":"))
                    push!(param,res)
                end
            end
        end
        return(param)
    end
end

_input_parameters (generic function with 1 method)

In [18]:
## parse input file

function _parse_input(input_param)
    let
        fileParam= "default.cfg" 
        saveFold=  "Results/"
        verbose= true
        
        for pair in input_param
            if pair[1] == "File_Parameters"
                fileParam = pair[2]
            elseif pair[1] == "Folder_Results"
                saveFold= pair[2]
            elseif pair[1] == "Display_Screen_Results" && pair[2] == "false"
                verbose= false
            end
        end
        res= _inputCfg(fileParam , saveFold , verbose)
        return(res)    
    end
end

_parse_input (generic function with 1 method)

In [19]:
## parse parameter file for the subarray constraints

function _parse_parameters(input_param)
    let
        arraycfg= 0 
        obs= 0
        sub= 0
        wei= 0
        ga= 0
        
        Array_Configuration_File= "alma.cfg"
        Result_Folder= "."
        Display_Verbose= true
        Observatory_Latitude= -23.0262015
        Source_Declination= -30
        Source_Hour_Angle = -1 
        Subarray_Number= 4
        
        ## array of nsubarray elements
        Pads_Per_Subarray= []
        Weight_Subarray= []
        Subarray_Name= [] 
        Spatial_Resolution= []
        Maximum_Recoverable_Scale=
        Elongation= []
        Sidelobe_Level= []
        Weight_Spatial_Resolution= []
        Weight_Maximum_Recoverable_Scale= [] 
        Weight_Elongation= []                    
        Weight_Sidelobe_Levels= []
        
        ## GA parameters
        Number_Iterations = 100
        Population_Size= 100
        Termination_Condition= false
        Threshold= -0.05
        Mutation_Rate= 0.05
        Tournament_Size= 5
        Number_Elitism= 5
        
        for pair in input_param
            println(pair)
            if pair[1] == "Array_Configuration_File"
                Array_Configuration_File= pair[2]
            elseif pair[1] == "Result_Folder"
                Folder_Results= pair[2]
            elseif pair[1] == "Display_Verbose"
                Display_Verbose=  pair[2]=="true" ? true : false
            elseif pair[1] == "Observatory_Latitude"
                Observatory_Latitude= parse(Float64, pair[2])    
            elseif pair[1] == "Source_Declination"
                Source_Declination= parse(Float64, pair[2])
            elseif pair[1] == "Source_Hour_Angle"
                Source_Hour_Angle= parse(Float64, pair[2])
            elseif pair[1] == Subarray_Number
                Subarray_Number= parse(Int, pair[2])
            elseif pair[1] == "Number_Iterations"
                Number_Iterations= parse(Int, pair[2])
            elseif pair[1] == "Population_Size"
                Population_Size= parse(Int, pair[2])
            elseif pair[1] == "Termination_Condition"
                Termination_Condition= pair[2]=="true" ? true : false
            elseif pair[1] == "Threshold"
                Threshold=parse(Float64, pair[2]) 
            elseif pair[1] == "Mutation_Rate"
                Mutation_Rate=parse(Float64, pair[2])
            elseif pair[1] == "Tournament_Size"
                Tournament_Size= parse(Int, pair[2])
            elseif pair[1] == "Number_Elitism"
                Number_Elitism= parse(Int, pair[2])  
            elseif pair[1] == "Pads_Per_Subarray"
                Pads_Per_Subarray= map(x->(v = tryparse(Int,x); 
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Weights_Subarray"
                  Weight_Subarray= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))  
            elseif pair[1] == "Subarray_Name" 
                Subarray_Name= split(pair[2],",")
            elseif pair[1] == "Spatial_Resolution"
                Spatial_Resolution= map(x->(v = tryparse(Float64,x); 
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Maximum_Recoverable_Scale"
                Maximum_Recoverable_Scale= map(x->(v = tryparse(Float64,x); 
                        v==nothing ? 0.0 : v),split(pair[2],","))       
            elseif pair[1] == "Elongation"
                Elongation= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Sidelobe_Level"
               Sidelobe_Level= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],",")) 
            elseif pair[1] == "Weight_Spatial_Resolution"
                Weight_Spatial_Resolution= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Weight_Maximum_Recoverable_Scale" 
                Weight_Maximum_Recoverable_Scale= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Weight_Elongation" 
                Weight_Elongation= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Weight_Sidelobe_Levels"
                Weight_Sidelobe_Levels=map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))
            end
        end
        
        ###
        println(typeof(Array_Configuration_File))
        
        arrcfg = CSV.read(convert(String,Array_Configuration_File), datarow=4 , header=["X" , "Y", "Z" , "diam" , "name"] , delim= " ")
           
        subrange = []
        start= 1
            for npad in Pads_Per_Subarray
                push!(subrange,start:start+npad-1)
                start= start+npad
            end
        
        ### setting the struct.
        
        obs= _observation(Array_Configuration_File, Result_Folder, Display_Verbose, Observatory_Latitude , Source_Declination,Source_Hour_Angle,
            Subarray_Number)
        sub= _subarrayParameters(Pads_Per_Subarray, Subarray_Name , subrange, Spatial_Resolution,
            Maximum_Recoverable_Scale , Elongation, Sidelobe_Level)
        wei= _weight(Weight_Subarray,Weight_Spatial_Resolution,Weight_Maximum_Recoverable_Scale, 
            Weight_Elongation,Weight_Sidelobe_Levels)
        ga= _GA(Number_Iterations , Population_Size,Termination_Condition ,Threshold ,
            Mutation_Rate , Tournament_Size ,Number_Elitism)
        
        
        ### population setting
        pop =_cfg(arrcfg , obs , sub , wei , ga)
        
   return(pop)     
end
end



_parse_parameters (generic function with 1 method)

In [20]:
function _check_consistency(pop::_cfg)
    nsubpads= sum(pop.sub.Pads_Per_Subarray)
    npads= nrow(pop.arr)
    if nsubpads != npads
        error("##Error: Number of pads per subarray is not equal to the total of pads.")
    end
    return(true)
end

## main function to read the cfg and check it.
function _read_cfg(inpfile, verbose=true)
    # res= _input_parameters(inpfile)
    # inpcfg= _parse_input(res)
    ## parameters inputs
    res= _input_parameters(inpfile)
    cfg= _parse_parameters(res)
    if verbose
        @printf("%3.3f", cfg.obs.Source_Declination)
    end
    _check_consistency(cfg)
    return(cfg)
end


_read_cfg (generic function with 2 methods)

In [21]:
macro main(inpfile)
    cfg= _read_cfg(inpfile)
end

@main("../master/data/GA_Parameters_random.txt.julia")
# @main("../master/data/GA_Parameters_O-10.txt.julia")

SubString{String}["Array_Configuration_File", "../master/data/random.cfg"]
SubString{String}["Result_Folder", "../products"]
SubString{String}["Display_Verbose", "true"]
SubString{String}["Observatory_Latitude", "-23.0262015"]
SubString{String}["Source_Declination", "-50"]
SubString{String}["Source_Hour_Angle", "0"]
SubString{String}["Subarray_Number", "5"]
SubString{String}["Pads_Per_Subarray", "30, 40, 10, 10 , 10"]
SubString{String}["Subarray_Name", "Sub_1, Sub_2 , Sub_3 , Sub_4 , Sub_5"]
SubString{String}["Spatial_Resolution", "1.0 , 2.0 , 3.5 , 3.5 , 5.0"]
SubString{String}["Maximum_Recoverable_Scale", "20 , 20 , 10 , 10.0 , 10.0"]
SubString{String}["Elongation", "1.2 , 1.3 , 2.0 , 3.0 , 4.0"]
SubString{String}["Sidelobe_Level", "10 , 20 , 60 , 60 , 60"]
SubString{String}["Weight_Spatial_Resolution", "0.25 , 0.25 ,  0.01, 0.01, 0.01"]
SubString{String}["Weight_Maximum_Recoverable_Scale", "0.02 , 0.02 , 0.01 , 0.01, 0.01"]
SubString{String}["Weight_Elongation", "0.25 , 0.2 , 0.01, 

_cfg(100×5 DataFrame
│ Row │ X        │ Y        │ Z        │ diam     │ name    │
│     │ Float64⍰ │ Float64⍰ │ Float64⍰ │ Float64⍰ │ String⍰ │
├─────┼──────────┼──────────┼──────────┼──────────┼─────────┤
│ 1   │ 236.338  │ 116.427  │ 0.298011 │ 12.0     │ X1      │
│ 2   │ 306.719  │ 245.051  │ 1.07687  │ 12.0     │ X2      │
│ 3   │ 226.495  │ 282.306  │ 0.738425 │ 12.0     │ X3      │
│ 4   │ 184.034  │ 203.345  │ 1.00067  │ 12.0     │ X4      │
│ 5   │ 317.61   │ 189.02   │ 0.689513 │ 12.0     │ X5      │
│ 6   │ 341.659  │ 245.022  │ 1.8598   │ 12.0     │ X6      │
│ 7   │ 80.2344  │ 77.0536  │ 1.0414   │ 12.0     │ X7      │
│ 8   │ 119.446  │ 340.428  │ 1.226    │ 12.0     │ X8      │
│ 9   │ 98.7349  │ 380.827  │ 0.289227 │ 12.0     │ X9      │
│ 10  │ 231.869  │ 318.016  │ 0.930413 │ 12.0     │ X10     │
⋮
│ 90  │ 333.745  │ 197.558  │ 1.99806  │ 12.0     │ X90     │
│ 91  │ 253.371  │ 129.706  │ 1.69844  │ 12.0     │ X91     │
│ 92  │ 135.146  │ 298.763  │ 0.385891 │ 12.0  